# Minimal Model Reasoning with Machine Learning Models
Code Implementation for Odyssey Project.

Fork of: https://github.com/porscheofficial/sls_sat_solving_with_deep_learning/blob/main/python/src/evaluate_with_given_params.py

Code adapted to use the GNN pre-trained Deep Learning Models as a SAT Solver, in order to Solve for Minimal Models.

# Details of Pre-Trained Models from Original Repository

In total, we provide three pre-trained models for the 3-SAT dataset and six pre-trained models for the pseudo-industrial datasets. They found in the folders `../../Data/models/random_3SAT/` and `../../Data/models/pseudo-industrial/`, respectively. The following table summarizes the characteristics of the models.

| Path of the pre-trained model        | Description of model                                                                                                    |
|----------------------------------|-----------------------------------------------------------------------------------------------------------------|
| `../../Data/models/random_3SAT/3_SAT_LLL.npy`| model trained on random 3-SAT using only the LLL-Loss     |
| `../../Data/models/random_3SAT/3_SAT_Gibbs.npy`|  model trained on random 3-SAT using only the Gibbs-Loss |
| `../../Data/models/random_3SAT/3_SAT_Gibbs_LLL.npy`|model trained on random 3-SAT using both the LLL-Loss and the Gibbs-Loss |
| `../../Data/models/pseudo_industrial/g4sat_easy_ca.npy`| model trained on easy CA instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_medium_ca.npy`| model trained on medium CA instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_hard_ca.npy`| model trained on hard CA instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_easy_ps.npy`| model trained on easy PS instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_medium_ps.npy`| model trained on medium PS instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_hard_ps.npy`| model trained on hard PS instances using only the Gibbs-Loss     |

The corresponding hyperparameters used for the training are specified in the corresponding config files in `../../experiments/configs/`.

In [ ]:
# Ensure that both paths are the same (change jupyter notebook's kernel if needed)
import sys
print(sys.executable)
!which python

In [ ]:
import os

def rename_dat_to_cnf(testpath, size_limit):
    counter = 0
    for file in os.listdir(testpath): 
        if file.endswith(".dat"):
            original_file = os.path.join(testpath, file)
            file_size = os.path.getsize(original_file)
            
            if (file_size < size_limit): # 2048, 51200, 102400 less than 100KB (1024 x 100 bytes)
                cnf_name = os.path.splitext(file)[0]
                new_file = os.path.join(testpath, f"{cnf_name}.cnf")
                print()
                print(f"{original_file}: {file_size} bytes")
                print(new_file)
                os.rename(original_file, new_file)
                counter += 1
                
    print(f"No. of Files Renamed: {counter}")

In [ ]:
import os
from data_utils import create_solutions_from_cnf

def prepare_cnf_data(testpath):
    # Create _sol.pkl files for each CNF file
    # check that function has been modified to maintain _sat.pkl naming convention regardless of SAT/UNSAT
    create_solutions_from_cnf(testpath)

    # Create a directory for each CNF file.
    # Move the CNF and respective _sol.pkl file into the respective directories.
    for file in os.listdir(testpath): 
        if file.endswith(".cnf"):
            print()
            print(os.path.join(testpath, file))
            
            cnf_name = os.path.splitext(file)[0]  # take only file name (without .cnf extension)
            cnf_folder = os.path.join(testpath, cnf_name)
            os.makedirs(cnf_folder, exist_ok=True)
            
            print(f"Created folder: {cnf_folder}")
            
            # Full original paths
            cnf_path = os.path.join(testpath, f"{cnf_name}.cnf")
            sol_path = os.path.join(testpath, f"{cnf_name}_sol.pkl")
            #print(cnf_path)
            #print(sol_path)

            os.rename(cnf_path, os.path.join(cnf_folder, f"{cnf_name}.cnf"))
            os.rename(sol_path, os.path.join(cnf_folder, f"{cnf_name}_sol.pkl"))
            
            print(f"Move CNF and solution files for {cnf_name} into {cnf_folder}")
    

In [ ]:
import os
import numpy as np
from evaluate_with_given_params import load_model_and_test

# Solve with pre-trained model
def solve_with_trained_model(cnf_file_path, model_path):
    print("Loading Model and Test...")
    results = load_model_and_test(
        data_path=cnf_file_path, # Directory containing CNF file(s)
        model_path=model_path,   # Pre-trained model to use
        n_steps=100,             # No. of steps
        n_runs=10,               # No. of runs
        algo_type="moser",       # "moser" for Moser-Tardos (MT) algorithm or "probsat" for WalkSAT algorithm
        path_save=False, 
        keep_traj=True,     
        pre_compute_mapping=True,
        prob_flip_best=0
    )
    
    # Interpret Results
    model_path, model_details, n_array, alpha_array, energies_array_mean, energies_array_median, total_steps = results
    
    # For Debugging
    #print(*results)
    #print(len(energies_array_mean))
    print(*energies_array_mean)
    
    print(f"No. of Variables: {n_array[0]}")
    print(f"Total Steps: {total_steps}")
    print(f"Last Step's Mean Energy: {energies_array_mean[-1]}")
    
    # Run x steps to find solution, take result from average energies of the last step (of 10 runs)
    # No. of Runs affect last array (no. of steps taken of each run). Mean energies is averaged out of all runs
    if energies_array_mean[-1] == 0:
        # Formula SATISFIABLE (can be solved)
        print("SAT")
        return True
    else:
        # Formula may be UNSATISFIABLE (cannot be solved) or needs more steps
        print("UNSAT / may require more steps")
        return False
        
    #return results

In [ ]:
from pysat.solvers import Solver, Minisat22
from pysat.formula import CNF

def compute_minimal_model(F):
    # Make mutable copy of F
    #print(F.clauses)
    working_formula = CNF(from_clauses = F.clauses)
    #print(working_formula)

    while True:
        with Minisat22(bootstrap_with = working_formula.clauses) as solver: # solver deleted after end of with block
            #print(solver.solve())
            if not (solver.solve()):
                return "UNSAT"

            model = solver.get_model()
            #print(model)
            model_set = set(model)
            #print(model_set)

            # If model is minimal, return the minimal model.
            if is_minimal(F, model_set):
                return model_set

            # Otherwise, model is not minimal
            # Block the current non-minimal model and try again
            # Add blocking clause to avoid repeating this model
            block_clause = block_model(model_set)
            print("Block Clause: ", block_clause)
            working_formula.append(block_clause)

def is_minimal(F, model_set):
    true_vars = set()
    false_vars = set()

    for lit in model_set:
        if lit > 0: # if literal is true
            true_vars.add(lit)
        else: # else literal is false
            false_vars.add(lit)

    #print("Model Set: ")
    #print(*model_set)
    #print("True Vars: ")
    #print(*true_vars)
    #print("False Vars: ")
    #print(*false_vars)
    
    # Make mutable copy of F
    minimal_check_formula = CNF(from_clauses = F.clauses)

    # Add unit clause for each false assignment (fix as false)
    for lit in false_vars:
        minimal_check_formula.append([lit])

    # Add disjunction to drop at least one true variable (change to false)
    if true_vars: # set is non-empty
        new_clause = [-v for v in true_vars]
        #print("New Clause: ", new_clause)
        minimal_check_formula.append(new_clause) # new_clause is a disjunction (OR) of literals
        
    else: # trivially minimal if no true variables (empty set is cast as boolean to false)
        return True
    
    #print("Minimal Check Formula: ")
    #print(*minimal_check_formula)
    
    # Traditional SAT Solver (to replace with Deep Learning Model)
    #solver = Minisat22(bootstrap_with = minimal_check_formula.clauses) 
    #result = solver.solve()
    #solver.delete()
    
    # Deep Learning SLS SAT Solver
    minimal_check_formula.to_file("../../Data/random_sat_data/extracted/output/minimal_check_formula.cnf")
    prepare_cnf_data("../../Data/random_sat_data/extracted/output/")
    
    print("\n INVOKING DEEP LEARNING SAT SOLVER \n ")
    # Change model accordingly 
    result = solve_with_trained_model(cnf_file_path="../../Data/random_sat_data/extracted/output/minimal_check_formula", 
                                      model_path="../../Data/models/pseudo_industrial/g4sat_hard_ps.npy")
    print("\n END OF INVOKING DEEP LEARNING SAT SOLVER \n ")
    print("Result from Deep Learning (is SAT): " + str(result))
    print("Result from Deep Learning (is minimal): " + str(not result) + "\n")
    
    # If result is true (SAT), then F is not minimal
    # Else result is false (UNSAT), then F is minimal.
    return not result

def block_model(model_set):
    # Only negate the literals assigned True (do not include negation of literals assigned False)
    # This blocks the model and all its supersets.
    clause = [-lit for lit in model_set if lit > 0]
    #print("Clause: ", clause)
    return clause

In [ ]:
# Examples (where variables a=1, b=2, c=3, d=4, e=5)

# F = (a ∨ b ∨ c), (¬a ∨ ¬b ∨ d), (¬a ∨ ¬b ∨ e)
cnf = CNF() # Conjunctive Normal Form
cnf.append([1, 2, 3])      # a ∨ b ∨ c
cnf.append([-1, -2, 4])    # ¬a ∨ ¬b ∨ d
cnf.append([-1, -2, 5])    # ¬a ∨ ¬b ∨ e
#cnf.append([6, 2, 3])    # f v b v c
minimal_model = compute_minimal_model(cnf)
print("Minimal model 1: ", minimal_model, "\n")
#cnf.to_file("../../Data/random_sat_data/mytest/example1.cnf")

# F = (¬a ∨ ¬b ∨ d), (¬c), (¬d)
cnf2 = CNF() # Conjunctive Normal Form
cnf2.append([-1, -2])
cnf2.append([-3])
cnf2.append([-4])
#print(*cnf2)
minimal_model2 = compute_minimal_model(cnf2)
print("Minimal model 2: ", minimal_model2, "\n")
#cnf2.to_file("../../Data/random_sat_data/mytest/example2.cnf")

# F = (a) AND (¬a)
cnf3 = CNF() # Conjunctive Normal Form
cnf3.append([1])      # a
cnf3.append([-1])    # ¬a
minimal_model3 = compute_minimal_model(cnf3)
print("Minimal model 3: ", minimal_model3, "\n")
#cnf3.to_file("../../Data/random_sat_data/mytest/example3.cnf")
    
print("==========================================")
print("PSEUDO-INDUSTRIAL CNF Formula Testing...")
print("==========================================")

# For each CNF file (CNF Formula), run the Deep Learning-based Minimal Model Solver.
target_directory = "../../Data/random_sat_data/extracted/"
rename_dat_to_cnf(target_directory, size_limit=2048)

for entry in os.listdir(target_directory):
    full_path = os.path.join(target_directory, entry)

    print("NEW ENTRY")
    if os.path.isfile(full_path) and entry.endswith(".cnf"):
        print(f"Found CNF file: {full_path}")
        print(f"Size: {os.path.getsize(full_path)} bytes")

        # For Minimal Model Solving
        cnf_formula = CNF(from_file=full_path)
        minimal_model = compute_minimal_model(cnf_formula)
        print("Minimal model: ", minimal_model)

    elif os.path.isdir(full_path):
        print(f"Found folder: {full_path}")

    print("=========================================\n\n")